In [97]:
from googleapiclient.discovery import build
from apiclient.errors import HttpError
from googletrans import Translator
import googleapiclient.errors
from dotenv import load_dotenv
from tqdm import tqdm
import pandas as pd
import numpy as np
import scrapetube
import requests
import time
import os
import io

In [2]:
translator = Translator()

In [ ]:
#creating a function that detects the source language and translates it to English

def translate(text):
    time.sleep(1)
    translator = Translator()
    translation = translator.translate(text, dest= 'en')
    return translation.text
    

In [9]:
#reading the previously downloaded main data
data = pd.read_csv('10_main_database.csv')
len(data)

2896


Testing out small batch on a copy df

In [5]:
data_copy = data.head(20).copy()

In [6]:
columns_to_translate = ['descriptions', 'titles', 'tags']

In [33]:
data_copy[['test1', 'test2', 'test3']] = data_copy[['descriptions', 'titles', 'tags']].map(translate)
data_copy.head(1)

,video_ids,descriptions,titles,view_counts,channel_ids,publish_dates,tags,thumbnails,captions,test1,test2,test3
0,VwnY9NK5K7A,【熊市不可錯過的幣圈資訊🔎 】 \n解放上班族！小資必學投資指南系列\nhttps://w...,馬上留言「+」領取 Friend.tech完整空投攻略👀#虛擬貨幣 #投資 #新手投資 #投...,437,UCD0G-1Q-3qawQEU2YpUvgfQ,2023-08-30T10:40:03Z,"['#每日幣研', '區塊鏈']",{'url': 'https://i.ytimg.com/vi/VwnY9NK5K7A/de...,False,[Information circle information that must not ...,"Leave a message ""+"" immediately to receive FRI...","['#Daily currency research', 'blockchain']"




Creating a copy df to work with and working with smaller items like titles for testing

In [34]:
data1 = data.copy()

In [ ]:
data1[['Translated_descriptions', 'Translated_title', 'Translated_tags']] = tqdm(data[['descriptions', 'titles', 'tags']].map(translate))

In [13]:
#code works for smaller chunks.
#here the code crashes due to sheer volume of data
for column in tqdm(columns_to_translate):
    data1[f'Translated {column}'] = data1[column].map(translate)
return data1.head()
    

  0%|                                                                                                                 | 0/3 [1:28:04<?, ?it/s]


ReadTimeout: The read operation timed out


Incorporating batches of rows to translate

In [39]:

#define a function to translate a batch of rows
def translate_batch(df, cols):
    translator = Translator()
    # create an empty dataframe to store the translations
    df_translated = pd.DataFrame()
    # loop through the columns to translate
    for col in cols:
        # apply the translator to the column and get the text attribute
        df_translated["Translated_"+ col] = df[col].apply(lambda x: translator.translate(x).text)
    # return the translated dataframe
    return df_translated

# define a function to translate the whole dataframe in batches
def translate_df(df, cols, batch_size, sleep_time):
    # create an empty list to store the batches
    batches = []
    # get the number of rows in the dataframe
    n_rows = len(df)
    # calculate the number of batches
    n_batches = (n_rows // batch_size) + 1
    # loop through the batches
    for i in range(n_batches):
        # get the start and end indices of the batch
        start = i * batch_size
        end = min((i + 1) * batch_size, n_rows)
        # get the batch dataframe
        df_batch = df.iloc[start:end]
        # translate the batch dataframe
        df_translated = translate_batch(df_batch, cols)
        # append the translated dataframe to the list
        batches.append(df_translated)
        # print the progress
        print(f"Translated batch {i + 1} of {n_batches}")
        # sleep for some time
        time.sleep(sleep_time)
    # concatenate the batches into one dataframe
    df_translated = pd.concat(batches, ignore_index=True)
    # join the translated dataframe with the original dataframe
    df_joined = df.join(df_translated)
    # return the joined dataframe
    return df_joined


In [41]:
#incurred timeout error after translating 17 batches
#have to break data into smaller chunks
cols_to_translate = ['titles', 'tags']
batch_size = 50
sleep_time = 1
dest = 'en'

trial = translate_df(data1, cols_to_translate, batch_size, sleep_time)
trial

Translated batch 1 of 58
Translated batch 2 of 58
Translated batch 3 of 58
Translated batch 4 of 58
Translated batch 5 of 58
Translated batch 6 of 58
Translated batch 7 of 58
Translated batch 8 of 58
Translated batch 9 of 58
Translated batch 10 of 58
Translated batch 11 of 58
Translated batch 12 of 58
Translated batch 13 of 58
Translated batch 14 of 58
Translated batch 15 of 58
Translated batch 16 of 58
Translated batch 17 of 58


ReadTimeout: The read operation timed out

Decided to translate captions separately due to sheer volume
Trying batches along with caption length limit

In [21]:
caption_df = pd.read_csv('caption_database_df.csv')
caption_df.head(1)

,video_id,caption_text
0,1NdOXAHzTHU,今天很開心來到香港 Ｗeb3 festival 有非常多國際級團隊聚集在一起 趁著這次機會 ...


In [ ]:
#creating function for just one caption

def translate_caption(caption, cap_size):
    #translator = Translator()
    translated_caption = ""
    if len(caption)>cap_size:
        for i in tqdm(range(0, len(caption), cap_size)):
            translation = translator.translate(caption[i: i + 150]).text
            translated_caption += translation
    else:
       translated_caption = translator.translate(caption).text
    return translated_caption  
            

In [32]:
# testing translate_caption for one caption
caption = caption_df.iloc[0]['caption_text']
trans = translate_caption(caption,150)
trans

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [00:04<00:00,  8.51it/s]


"Today, I am very happy to come to Hong Kong Web3 Festival. There are many international teams gathered together to take advantage of this opportunity. We will visit the well -known wallet IMTOKEN top public chain Avalanche and the largest blockchain from Malaysia, let us start!Hi everyone, this time, we are also very happy to interview Jiadong from IMTOKEN to follow meWe answered some of his gains about this web3 Festival and some of the progress directions of the IMTOKEN's follow -up team. Then do you think of this activity? Far from Singapore, then from the perspective of IMTOKENWe come to participate in this event this time mainly have two purposes. In the first place, our headquarters is in Singapore. We have also been deeply cultivated in Singapore for a long time. In the process of applying for a license, we have come to such a big one because Singapore is actually a relatively large layout in the past few years.We found that there is a relatively large wind direction change in 

In [70]:

#creating function that can iterate through each caption from a list of cols
#output = list of translated captions

def translate_caption(df, cols, cap_size):
    #translator = Translator()
    translated_captions = []
    
    for col in cols:
        for i in range(len(df)):
            translated_caption = ''
            caption = df.iloc[i][col]
            
            if len(caption)>cap_size:
                for i in tqdm(range(0, len(caption), cap_size)):
                    translation = translator.translate(caption[i: i + cap_size]).text
                    translated_caption += translation
            else:
                translated_caption = translator.translate(caption)
            
            translated_captions.append(translated_caption)
    return translated_captions
            

In [71]:
#testing with just 2 rows
test_df = caption_df.head(2)
cols = ['caption_text']
test = translate_caption(test_df,cols , 150)
test

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:04<00:00,  7.15it/s]


["Today, I am very happy to come to Hong Kong Web3 Festival. There are many international teams gathered together to take advantage of this opportunity. We will visit the well -known wallet IMTOKEN top public chain Avalanche and the largest blockchain from Malaysia, let us start!Hi everyone, this time, we are also very happy to interview Jiadong from IMTOKEN to follow meWe answered some of his gains about this web3 Festival and some of the progress directions of the IMTOKEN's follow -up team. Then do you think of this activity? Far from Singapore, then from the perspective of IMTOKENWe come to participate in this event this time mainly have two purposes. In the first place, our headquarters is in Singapore. We have also been deeply cultivated in Singapore for a long time. In the process of applying for a license, we have come to such a big one because Singapore is actually a relatively large layout in the past few years.We found that there is a relatively large wind direction change in

In [ ]:

#creating function that can iterate through each caption from a list of cols
#output = list of translated captions
#creating function that can iterate through each caption from a list of cols
#output = dict of video_id along with the corresponding translated captions
def translate_caption2(df, cols, cap_size):
    if 'video_id' in df.columns:
        df = df.set_index('video_id')
    #translator = Translator()
    translated_captions = {}
    
    for col in cols:
        col_df = df[col]
        for video_id, caption in col_df.iterrows():
            translated_caption = ''            
            if len(caption)>cap_size:
                for i in tqdm(range(0, len(caption), cap_size)):
                    translation = translator.translate(caption[i: i + cap_size]).text
                    translated_caption += translation
            else:
                translated_caption = translator.translate(caption)
            
            translated_captions[video_id] = translated_caption # use the video_id as the key and the translated_caption as the value
    return translated_captions
            

In [73]:
#creating function that can iterate through each caption from a list of cols
#output = dict of video_id along with the corresponding translated captions

def translate_caption(df, cols, cap_size):
    #translator = Translator()
    translated_captions = {}
    for col in cols:
        for i in range(len(df)):
            translated_caption = ''
            caption = df.iloc[i][col]
            video_id = df.iloc[i]['video_id']
            
            if len(caption)>cap_size:
                for i in tqdm(range(0, len(caption), cap_size)):
                    translation = translator.translate(caption[i: i + 150]).text
                    translated_caption += translation
            else:
                translated_caption = translator.translate(caption)
            
            translated_captions[video_id] = translated_caption # use the video_id as the key and the translated_caption as the value
    return translated_captions



In [74]:
#testing with just 2 rows
test_df2 = caption_df.head(2)
cols = ['caption_text']
test2 = translate_caption(test_df,cols , 150)
test2

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:08<00:00,  3.75it/s]


{'1NdOXAHzTHU': "Today, I am very happy to come to Hong Kong Web3 Festival. There are many international teams gathered together to take advantage of this opportunity. We will visit the well -known wallet IMTOKEN top public chain Avalanche and the largest blockchain from Malaysia, let us start!Hi everyone, this time, we are also very happy to interview Jiadong from IMTOKEN to follow meWe answered some of his gains about this web3 Festival and some of the progress directions of the IMTOKEN's follow -up team. Then do you think of this activity? Far from Singapore, then from the perspective of IMTOKENWe come to participate in this event this time mainly have two purposes. In the first place, our headquarters is in Singapore. We have also been deeply cultivated in Singapore for a long time. In the process of applying for a license, we have come to such a big one because Singapore is actually a relatively large layout in the past few years.We found that there is a relatively large wind dire

In [100]:
#testing with just 2 rows
test_df5= caption_df
cols = ['caption_text']
test2 = translate_caption(test_df,cols , 50)
test2

 16%|████████████████▏                                                                                       | 19/122 [00:10<00:59,  1.73it/s]


AttributeError: 'Translator' object has no attribute 'raise_Exception'

In [103]:
# finale 

def translate_caption_batch(dataframe, columns, caption_size, batch_size, sleep_time):
    batches = []
    n_rows = len(dataframe)
    n_batches = (n_rows//batch_size)+1
    
    for i in tqdm(range(n_batches)):
        start = i* batch_size
        end = min((i+1)* batch_size, n_rows)
        batch_df = dataframe.loc[start:end]
        caption_size = caption_size

        try:
            translated_dict = translate_caption(batch_df, columns, caption_size)
            translated_df = pd.DataFrame(translated_dict)
            batches.append(translated_df)
        except Exception as e:
            print(f"Error occurred while translating batch {i + 1}: {e}")
        
        print(f"Translated batch {i + 1} of {n_batches}")
        time.sleep(sleep_time)
    # concatenate the batches into one dataframe
    df_translated = pd.concat(batches, ignore_index=True)
    # join the translated dataframe with the original dataframe
    df_joined = dataframe.merge(df_translated, on = 'video_id')
    # return the joined dataframe
    return df_joined
            

In [81]:
caption_df_copy = caption_df.copy()

In [ ]:
dataframe = caption_df_copy
columns = ['caption_text']
batch_size = 30
sleep_time = 10
caption_size = 150
Test_main = translate_caption_batch(dataframe, columns, caption_size, batch_size, sleep_time)
Test_main

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [00:04<00:00,  8.45it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:03<00:00,  8.69it/s]

 63%|██████████████████████████████████████████████████████████████████▌                                      | 19/30 [00:07<00:04,  2.69it/s]


Error occurred while translating batch 1: The read operation timed out
Translated batch 1 of 20


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 51/51 [00:46<00:00,  1.09it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 38/38 [00:35<00:00,  1.07it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:29<00:00,  1.01it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:26<00:00,  1.14it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:29<00:00,  1.10it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:28<00:00,  1.07s/it]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:25<00:00,  1.1

Error occurred while translating batch 2: If using all scalar values, you must pass an index
Translated batch 2 of 20


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 31/31 [00:03<00:00,  9.53it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:21<00:00,  1.07it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:31<00:00,  1.15it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:19<00:00,  1.40it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:22<00:00,  1.18it/s]

  7%|███████▊                                                                                                  | 2/27 [00:06<01:20,  3.21s/it]


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 92/92 [01:17<00:00,  1.19it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:15<00:00,  1.26it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:14<00:00,  1.25it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:16<00:00,  1.06it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:16<00:00,  1.16it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:20<00:00,  1.17it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:29<00:00,  1.1

Error occurred while translating batch 13: The read operation timed out
Translated batch 13 of 20


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:12<00:00,  1.23it/s]

 52%|██████████████████████████████████████████████████████▍                                                  | 14/27 [00:15<00:14,  1.08s/it]


Error occurred while translating batch 14: The read operation timed out
Translated batch 14 of 20


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:31<00:00,  1.06it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:20<00:00,  1.10it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:18<00:00,  1.14it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:18<00:00,  1.13it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:25<00:00,  1.05it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:28<00:00,  1.01it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:15<00:00,  1.5